# Part I. ETL Pipeline for Pre-Processing the Files

#### Importing Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)
    #break

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra Modeling and Queries

## The CSV file titled event_datafile_new.csv, generated above, contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the **event_datafile_new.csv**:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Creating Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Setting Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Query 1:  Gives artist, song title and song's length in the music app history for a given sessionId and itemInSession

### Since the query is asking for __artist name__, __song title__ and __song's length__ those columns needs to be in the table. The query also wants to filter by __session id__ and __item in session__ so those columns needs to be in the primary key. The primary key is formed by __session id__ and __item in session__ where the first is the partition key and the second is a clustering key. That combination is also sufficient to make each row key unique. At the end, the table is as follows:

```
Table Name: artist_song_details
column 1: session_id int
column 2: item_session int 
column 3: artist_name text
Column 4: song_title text
Column 5: song_length decimal
PRIMARY KEY(session_id, item_session)
```

As an example we look for sessionId = 338 and itemInSession = 4

In [8]:
query = "CREATE TABLE IF NOT EXISTS artist_song_details "
query = query + "(session_id int, item_session int, artist_name text, song_title text, song_length decimal, \
                  PRIMARY KEY (session_id, item_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [9]:
file = 'event_datafile_new.csv'

col = {}
col['artist'] = 0
col['first_name'] = 1
col['gender'] = 2
col['item_session'] = 3
col['last_name'] = 4
col['length'] = 5
col['level'] = 6
col['location'] = 7
col['session_id'] = 8
col['song'] = 9
col['user_id'] = 10

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)    
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_details (session_id, item_session, artist_name, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[col['session_id']]), int(line[col['item_session']]), line[col['artist']], line[col['song']], float(line[col['length']])))

In [10]:
query = "SELECT artist_name, song_title, song_length \
         FROM artist_song_details \
         WHERE session_id=338 AND item_session=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.3073


### Query 2: Gives name of artist, song (sorted by itemInSession) and user (first and last name) in the music app history for a given userId and sessionId

### Since the query is asking for __artist name__, __song title__ (sorted by __item in session__), __user first__ and __last name__ those columns needs to be in the table, while __item in session__ needs to be a clustering column in the primary key in order to do the sorting. The query also wants to filter by __user id__ and __session id__ so those columns needs to be in the primary key. The partition key is formed by __user id__ and __session id__ composed while __item in session__ is a clustering key. That combination is also sufficient to make each row key unique. At the end, the table is as follows:

```
Table Name: artist_song_user
column 1: user_id int
column 2: session_id int 
Column 3: item_session int
column 4: artist_name text
Column 5: song_title text
Column 6: user_first_name text
Column 7: user_last_name text
PRIMARY KEY((user_id, session_id), item_session)
```

As an example, we look for userid = 10 and sessionid = 182

In [11]:
query = "CREATE TABLE IF NOT EXISTS artist_song_user "
query = query + "(user_id int, session_id int, item_session int, artist_name text, song_title text, user_first_name text, user_last_name text, \
                 PRIMARY KEY ((user_id, session_id), item_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)


In [12]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)    
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_user (user_id, session_id, item_session, artist_name, song_title, user_first_name, user_last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[col['user_id']]), int(line[col['session_id']]), int(line[col['item_session']]), \
                                line[col['artist']], line[col['song']], line[col['first_name']], line[col['last_name']]))
                    

In [13]:
query = "SELECT artist_name, song_title, user_first_name, user_last_name \
         FROM artist_song_user \
         WHERE user_id=10 AND session_id=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.user_first_name, row.user_last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query 3: Gives every user name (first and last) in the music app history who listened to a song.

### Since the query is asking for __user first__ and __last name__ those columns needs to be in the table. The query also wants to filter by __song title__ so this column needs to be in the primary key. In order to make each row key unique, it is necessary to also have in the primary key the column __user id__. The primary key is formed by __song title__ and __user id__. At the end, the table is as follows:

```
Table Name: song_user_history
column 1: song_title text
column 2: user_id int 
Column 3: user_first_name text
Column 4: user_last_name text
PRIMARY KEY(song_title, user_id)
```

As an example, we look at all users who listened to the song 'All Hands Against His Own'

In [14]:
query = "CREATE TABLE IF NOT EXISTS song_user_history "
query = query + "(song_title text, user_id int, user_first_name text, user_last_name text, \
                 PRIMARY KEY (song_title, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)


In [15]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)    
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_user_history (song_title, user_id, user_first_name, user_last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[col['song']], int(line[col['user_id']]), \
                                line[col['first_name']], line[col['last_name']]))

In [16]:
query = "SELECT user_first_name, user_last_name \
         FROM song_user_history \
         WHERE song_title='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user_first_name, row.user_last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


#### Droping tables and closing out the sessions

In [17]:
query = "DROP TABLE IF EXISTS artist_song_details"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [18]:
query = "DROP TABLE IF EXISTS artist_song_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [19]:
query = "DROP TABLE IF EXISTS song_user_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

#### Closing the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()